### Deliverable 1: Preprocessing the Data for a Neural Network

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

### Optimization Attempt 1: Re-bin Affilication and Classification columns

In [2]:
# Import and read the charity_data.csv.
application_df_1 = pd.read_csv("Resources/charity_data.csv")
application_df_1.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_1.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [4]:
# Determine the number of unique values in each column.
application_df_1.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Get AFFILIATION value counts
application_df_1['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [6]:
# Create criteria for AFFILIATION replacement
aff_values = application_df_1['AFFILIATION'].value_counts()
replace_affiliation = [aff for aff in aff_values.index if (aff_values[aff] < 100)]

# Replace in dataframe
for app in replace_affiliation:
    application_df_1.AFFILIATION = application_df_1.AFFILIATION.replace(app,"Other")
    
# Check to make sure binning was successful
application_df_1.AFFILIATION.value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [7]:
# Create criteria for APPLICATION_TYPE replacement
app_values = application_df_1['APPLICATION_TYPE'].value_counts()

replace_application = [app for app in app_values.index if (app_values[app] < 500)]

# Replace in dataframe
for app in replace_application:
    application_df_1.APPLICATION_TYPE = application_df_1.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df_1.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [8]:
# Determine which values to replace if counts are less than ..?
class_values = application_df_1['CLASSIFICATION'].value_counts()
replace_class = [cls for cls in class_values.index if (class_values[cls] < 700)]

# Replace in dataframe
for cls in replace_class:
    application_df_1.CLASSIFICATION = application_df_1.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_1.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [9]:
# Generate our categorical variable lists
app_cat = application_df_1.dtypes[application_df_1.dtypes == 'object'].index.tolist()
app_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_1[app_cat], 1))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [11]:
# Merge one-hot encoded features and drop the originals
application_df_1 = application_df_1.merge(encode_df, left_index=True, right_index=True)
application_df_1 = application_df_1.drop(app_cat, 1)
application_df_1.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [12]:
# Split our preprocessed data into our features and target arrays
X = application_df_1.drop(['IS_SUCCESSFUL', 'SPECIAL_CONSIDERATIONS_N'], 1)
y = application_df_1['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [13]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_inputs = len(X_train_scaled[0])
hidden_nodes_1 = 90
hidden_nodes_2 = 60

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_1,
    activation='relu',
    input_dim=num_inputs
))

# Second hidden layer
nn.add(tf.keras.layers.Dense(
    units=hidden_nodes_2,
    activation='relu'
))

# Output layer
nn.add(tf.keras.layers.Dense(
    units=1,
    activation='sigmoid'
))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 90)                3690      
_________________________________________________________________
dense_1 (Dense)              (None, 60)                5460      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 61        
Total params: 9,211
Trainable params: 9,211
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Create callback function to keep checkpoints
import os
from tensorflow.keras.callbacks import ModelCheckpoint

os.makedirs('checkpoints/', exist_ok=True)
checkpoint_path = 'checkpoints/weights.{epoch:02d}.hdf5'

cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    period=5
)

# Compile the model
nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5698 - accuracy: 0.7189
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5557 - accuracy: 0.7274
Epoch 3/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5523 - accuracy: 0.7303
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5509 - accuracy: 0.7301
Epoch 5/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5491 - accuracy: 0.7321

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5479 - accuracy: 0.7318
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5474 - accuracy: 0.7341
Epoch 8/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5464 - accuracy: 0.7329
Epoch 9/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5466 - accuracy: 0.7324
Epoch 10/50
804/804 [=====================

In [17]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7269
Loss: 0.5545588135719299, Accuracy: 0.7268804907798767


In [ ]:
nn.save()

### Optimization Attempt 2: Add additional hidden layer

In [246]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_inputs = len(X_train_scaled[0])
hidden_nodes_1 = 90
hidden_nodes_2 = 40

nn_2 = tf.keras.models.Sequential()

# First hidden layer
nn_2.add(tf.keras.layers.Dense(
    units=hidden_nodes_1,
    activation='relu',
    input_dim=num_inputs
))

# Second hidden layer
nn_2.add(tf.keras.layers.Dense(
    units=hidden_nodes_2,
    activation='relu'
))

# Third hidden layer
nn_2.add(tf.keras.layers.Dense(
    units=hidden_nodes_2,
    activation='relu'
))

# Fourth hidden layer
nn_2.add(tf.keras.layers.Dense(
    units=hidden_nodes_2,
    activation='relu'
))

# Output layer
nn_2.add(tf.keras.layers.Dense(
    units=1,
    activation='sigmoid'
))

# Check the structure of the model
nn_2.summary()

# Compile the model
nn_2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 90)                3690      
_________________________________________________________________
dense_68 (Dense)             (None, 40)                3640      
_________________________________________________________________
dense_69 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_70 (Dense)             (None, 40)                1640      
_________________________________________________________________
dense_71 (Dense)             (None, 1)                 41        
Total params: 10,651
Trainable params: 10,651
Non-trainable params: 0
_________________________________________________________________


In [247]:
# Train the model
fit_model = nn_2.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5658 - accuracy: 0.7206
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5497 - accuracy: 0.7333

Epoch 00002: saving model to checkpoints/weights.02.hdf5
Epoch 3/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5470 - accuracy: 0.7336
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5448 - accuracy: 0.7353
Epoch 5/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5444 - accuracy: 0.7355
Epoch 6/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5430 - accuracy: 0.7360: 0s - loss: 0.5435 - ac
Epoch 7/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5417 - accuracy: 0.7371

Epoch 00007: saving model to checkpoints/weights.07.hdf5
Epoch 8/50
804/804 [==============================] - 1s 2ms/step - loss: 0.5410 - accuracy: 0.7378
Epoch 9/50
804/804 [==============================] - 1s 2ms

In [248]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5737 - accuracy: 0.7171
Loss: 0.5737301111221313, Accuracy: 0.717084527015686


### Optimization Attempt 3: Remove features

In [18]:
# Import and read the charity_data.csv.
application_df_3 = pd.read_csv("Resources/charity_data.csv")
application_df_3.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [19]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_3.drop(['EIN', 'NAME'], axis=1, inplace=True)

In [20]:
# Get value counts of USE_CASE
application_df_3['USE_CASE'].value_counts()

Preservation     28095
ProductDev        5671
CommunityServ      384
Heathcare          146
Other                3
Name: USE_CASE, dtype: int64

In [21]:
# Get value counts of SPECIAL_CONSIDERATIONS
application_df_3['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [22]:
# Remove USE_CASE, and SPECIAL_CONSIDERATIONS
application_df_3.drop(
    ['USE_CASE', 'SPECIAL_CONSIDERATIONS'],
    axis=1,
    inplace=True
)

In [23]:
# Create criteria for APPLICATION_TYPE replacement
app_values = application_df_3['APPLICATION_TYPE'].value_counts()

replace_application = [app for app in app_values.index if (app_values[app] < 500)]

# Replace in dataframe
for app in replace_application:
    application_df_3.APPLICATION_TYPE = application_df_3.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df_3.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [24]:
# Create criteria to replace CLASSIFICATION values
class_values = application_df_3['CLASSIFICATION'].value_counts()
replace_class = [cls for cls in class_values.index if (class_values[cls] < 700)]

# Replace in dataframe
for cls in replace_class:
    application_df_3.CLASSIFICATION = application_df_3.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df_3.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [25]:
# Generate our categorical variable lists
app_cat = application_df_3.dtypes[application_df_3.dtypes == 'object'].index.tolist()
app_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'ORGANIZATION',
 'INCOME_AMT']

In [26]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df_3[app_cat], 1))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Merge one-hot encoded features and drop the originals
application_df_3 = application_df_3.merge(encode_df, left_index=True, right_index=True)
application_df_3 = application_df_3.drop(app_cat, 1)
application_df_3.head(10)

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,1,5000,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,1,31452,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,1,7508025,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
8,1,94389,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# Split our preprocessed data into our features and target arrays
X = application_df_3.drop(['IS_SUCCESSFUL'], 1)
y = application_df_3['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

In [29]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_inputs = len(X_train_scaled[0])
hidden_nodes_1 = 80
hidden_nodes_2 = 30

nn_3 = tf.keras.models.Sequential()

# First hidden layer
nn_3.add(tf.keras.layers.Dense(
    units=hidden_nodes_1,
    activation='relu',
    input_dim=num_inputs
))

# Second hidden layer
nn_3.add(tf.keras.layers.Dense(
    units=hidden_nodes_2,
    activation='relu'
))

# output layer
nn_3.add(tf.keras.layers.Dense(
    units=1,
    activation='sigmoid'
))

# Check the structure of the model
nn_3.summary()

# Compile the model
nn_3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 80)                3040      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 5,501
Trainable params: 5,501
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Train the model
fit_model = nn_3.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5713 - accuracy: 0.7187
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5576 - accuracy: 0.7267
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7276
Epoch 4/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5544 - accuracy: 0.7282: 1s - loss: 0.5479 - accu
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5526 - accuracy: 0.7293

Epoch 00005: saving model to checkpoints/weights.05.hdf5
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5532 - accuracy: 0.7290
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7303: 0s - loss: 0.5530 - 
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5513 - accuracy: 0.7297
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5501 - accuracy: 0

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5534 - accuracy: 0.7318
Loss: 0.5534055233001709, Accuracy: 0.7317784428596497


In [33]:
# Export model
nn_3.save('AlphabetSoupCharity_Optimization.h5')